In [82]:
# Import all necessery python packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import re
import requests as req
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import time
import seaborn as sns

import patsy
import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression, Lasso, LassoCV, RidgeCV, Ridge, lars_path
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.pipeline import Pipeline

from IPython.display import Image

import pickle

import folium

import geojson

from IPython.display import display

import geopandas as gpd

In [83]:
def loadpickle(filename):
    with open(filename, 'rb') as f:
        return pickle.load(f)

In [84]:
df = loadpickle('./data/dataframe.pkl')

In [85]:
df_work = df.copy()

In [86]:
df_work.head()

,ID,SIZE,PRICE,ROOMS,FLOOR,ADDRESS,QUALITY,YEAR_BUILT,FLOORS_IN_BUILDING,LIFT,Balcony,ORIENTATION,VIEW,METRO_NEARBY,PARKING,DISTRICT,LATITUDE,LONGITUDE
0,23545892,39 m²,"31,9 millió Ft",1,földszint,"7. kerület, Dob utca 50.",közepes állapotú,1950 előtt,2,nincs megadva,NaN,dél,utcai,False,"utca, közterület - fizetős övezet",7,47.501126,19.065041
1,23652121,79 m²,"65,9 millió Ft",3,4,"6. kerület, Belső-Terézváros",felújított,1950 előtt,nincs megadva,nincs megadva,NaN,nincs megadva,nincs megadva,False,nincs megadva,6,47.509494,19.065323
2,23662271,75 m²,89 millió Ft,2 + 1 fél,1,"7. kerület, Belső-Erzsébetváros",újszerű,1950 előtt,4,van,NaN,délnyugat,utcai,False,nincs megadva,7,47.502627,19.077243
3,23662566,50 m²,"22,9 millió Ft",2,4,"3. kerület, Bécsi út",felújítandó,1950-1980 között,10,van,NaN,nyugat,utcai,False,"utca, közterület - fizetős övezet",3,47.532673,19.036491
4,23688806,32 m²,"12,5 millió Ft",1,3,"9. kerület, Haller utca",jó állapotú,NaN,3,nincs,NaN,nincs megadva,nincs megadva,False,"utca, közterület",9,47.476099,19.077198


In [6]:
df_work['SIZE'] = df['SIZE'].str[:-3].str.replace(" ","").astype('int')

In [7]:
df_work['SIZE']

0         39
1         79
2         75
3         50
4         32
5         75
6         77
7         45
8        126
9         62
10        48
11        70
12       108
13        95
14        95
15        95
16        65
17        85
18       110
19        59
20        35
21        85
22       167
23       136
24        72
25        96
26        95
27       131
28        89
29       140
        ... 
18531     41
18532     53
18533     42
18534     71
18535     22
18536     36
18537     30
18538     78
18539     35
18540     38
18541    107
18542     57
18543     40
18544     73
18545     81
18546     86
18547     58
18548     91
18549     36
18550     71
18551     89
18552     32
18553     39
18554     42
18555    132
18556     24
18557     90
18558     68
18559     84
18560     30
Name: SIZE, Length: 18561, dtype: int64

In [8]:
milliard_df = df_work[(df_work['PRICE'].str.endswith('milliárd Ft', na=False))]
millio_df = df_work[(df_work['PRICE'].str.endswith('millió Ft', na=False))]

In [9]:
df_work.loc[df_work['PRICE'].str.endswith('milliárd Ft', na=False), 'PRICE'] = milliard_df['PRICE'].str.split().str[0].str.replace(',', '.').astype('float') * 1000
df_work.loc[df_work['PRICE'].str.endswith('millió Ft', na=False), 'PRICE'] = millio_df['PRICE'].str.split().str[0].str.replace(',', '.').astype('float')

In [10]:
df_work['rooms'] = df_work['ROOMS']
df_work['half_rooms'] = 0

In [11]:
df_work[['rooms', 'half_rooms']] = df_work['ROOMS'].str[:-4].str.replace(" ","").str.split('\+',expand=True)

In [12]:
df_work.loc[~(df_work['ROOMS'].str.contains('fél', na=False)), 'rooms'] = df_work[~(df_work['ROOMS'].str.contains('\+', na=False))]['ROOMS']
#df_work['rooms'] = df_work['ROOMS']

In [13]:
df_work['half_rooms'].fillna(value=0, inplace=True)

In [14]:
df_work['rooms'] = df_work['rooms'].str.replace(" ","").astype('int')

In [15]:
df_work[df_work['rooms']>10 ]

,ID,SIZE,PRICE,ROOMS,FLOOR,ADDRESS,QUALITY,YEAR_BUILT,FLOORS_IN_BUILDING,LIFT,Balcony,ORIENTATION,VIEW,METRO_NEARBY,PARKING,DISTRICT,LATITUDE,LONGITUDE,rooms,half_rooms
97,25441982,1020,1200,52,nincs megadva,"7. kerület, Király utca",nincs megadva,NaN,nincs megadva,nincs megadva,NaN,nincs megadva,utcai,False,teremgarázs hely - benne van az árban,7,47.506331,19.070639,52,0
667,27935639,140,89.9,11,földszint,"6. kerület, Külső-Terézváros",jó állapotú,NaN,nincs megadva,van,NaN,délnyugat,utcai,False,nincs megadva,6,47.509494,19.065323,11,0
1160,28706021,351,239.9,12,1,"12. kerület, Városmajor utca",jó állapotú,1950 előtt,3,nincs,26 m²,nincs megadva,kertre néző,False,"utca, közterület",12,47.504458,19.019731,12,0
1419,28946876,825,580,30,4,"7. kerület, Damjanich utca",újszerű,2001-2010 között,4,van,26 m²,nyugat,utcai,False,"utca, közterület",7,47.506144,19.074918,30,0
2085,29325197,345,450,12,3,"7. kerület, Belső-Erzsébetváros",felújított,2019,nincs megadva,van,NaN,nincs megadva,nincs megadva,False,nincs megadva,7,47.502627,19.077243,12,0
5708,30386137,360,450,12,2,"7. kerület, Dohány utca",nincs megadva,NaN,5,van,NaN,nincs megadva,utcai,False,nincs megadva,7,47.500448,19.073166,12,0
6269,30470449,700,699,40,földszint,"5. kerület, Váci utca",közepes állapotú,1950-1980 között,3,nincs megadva,NaN,nincs megadva,nincs megadva,False,nincs megadva,5,47.493201,19.052852,40,0
10541,30933370,2222222,222220,222,nincs megadva,"7. kerület, Rákóczi út",nincs megadva,NaN,nincs megadva,nincs megadva,222219 m²,nincs megadva,nincs megadva,False,nincs megadva,7,47.499044,19.077575,222,0
11055,30969868,835,799,14,nincs megadva,"2. kerület, Rózsadomb",nincs megadva,1950 előtt,nincs megadva,van,90 m²,nincs megadva,nincs megadva,False,nincs megadva,2,47.517329,19.028288,14,0
12787,31087012,222,999,11 + 18 fél,nincs megadva,"1. kerület, Krisztina tér",nincs megadva,NaN,nincs megadva,nincs megadva,22 m²,nincs megadva,nincs megadva,False,nincs megadva,1,47.496863,19.031734,11,18


In [16]:
df_work[(df_work['ROOMS'].str.contains('\+', na=False))]

,ID,SIZE,PRICE,ROOMS,FLOOR,ADDRESS,QUALITY,YEAR_BUILT,FLOORS_IN_BUILDING,LIFT,Balcony,ORIENTATION,VIEW,METRO_NEARBY,PARKING,DISTRICT,LATITUDE,LONGITUDE,rooms,half_rooms
2,23662271,75,89,2 + 1 fél,1,"7. kerület, Belső-Erzsébetváros",újszerű,1950 előtt,4,van,NaN,délnyugat,utcai,False,nincs megadva,7,47.502627,19.077243,2,1
8,23792850,126,169.9,3 + 1 fél,4,"5. kerület, Zrínyi utca",felújítandó,NaN,6,van,6 m²,délnyugat,panorámás,False,"utca, közterület",5,47.500191,19.048434,3,1
16,23936899,65,8.65,1 + 2 fél,földszint,"9. kerület, Üllői út",közepes állapotú,1950 előtt,3,nincs,NaN,délkelet,utcai,False,"utca, közterület - fizetős övezet",9,47.473919,19.103220,1,2
21,24171833,85,106.29,1 + 1 fél,nincs megadva,"2. kerület, Budakeszi út",újszerű,2001-2010 között,6,van,10 m²,délnyugat,panorámás,False,teremgarázs hely - benne van az árban,2,47.517581,18.995488,1,1
32,24406258,69,69.9,2 + 1 fél,2,"6. kerület, Zichy Jenő utca",felújított,2019,5,van,10.11 m²,nincs megadva,utcai,False,"utca, közterület",6,47.506963,19.060848,2,1
36,24464188,159,380.87,2 + 2 fél,5,"5. kerület, Lipótváros",újszerű,2001-2010 között,7,van,NaN,kelet,nincs megadva,False,teremgarázs hely,5,47.504543,19.050380,2,2
42,24612503,81,45,2 + 1 fél,nincs megadva,"7. kerület, Külső-Erzsébetváros",nincs megadva,NaN,nincs megadva,nincs megadva,NaN,nincs megadva,nincs megadva,False,"utca, közterület",7,47.502627,19.077243,2,1
45,24657383,83,85,1 + 3 fél,földszint,"3. kerület, Testvérhegy",újszerű,2015,nincs megadva,nincs,16 m²,kelet,kertre néző,False,önálló garázs - benne van az árban,3,47.563934,19.013587,1,3
49,24802343,75,59,2 + 1 fél,3,"6. kerület, Aradi utca",újszerű,1950 előtt,3,nincs megadva,NaN,nincs megadva,nincs megadva,False,nincs megadva,6,47.506118,19.062051,2,1
59,24970755,57,30.2,1 + 2 fél,nincs megadva,"3. kerület, Jendrassik György utca",felújított,NaN,nincs megadva,van,NaN,nincs megadva,nincs megadva,False,nincs megadva,3,47.599829,19.048143,1,2


In [17]:
df_work['FLOOR'].unique()

array(['földszint', '4', '1', '3', 'nincs megadva', '2', '5', '7', '8',
       'félemelet', 'szuterén', '6', '9', '10', '10 felett'], dtype=object)

In [18]:
df_work.loc[(df_work['FLOOR']=='földszint'), 'FLOOR'] = 0
df_work.loc[(df_work['FLOOR']=='félemelet'), 'FLOOR'] = 1
df_work.loc[(df_work['FLOOR']=='szuterén'), 'FLOOR'] = -1
df_work.loc[(df_work['FLOOR']=='10 felett'), 'FLOOR'] = 11
df_work.loc[(df_work['FLOOR']=='nincs megadva'), 'FLOOR'] = 0
df_work['FLOOR'] = df_work['FLOOR'].astype('int')

In [19]:
df_work['QUALITY'].unique() 

array(['közepes állapotú', 'felújított', 'újszerű', 'felújítandó',
       'jó állapotú', 'új építésű', 'nincs megadva', 'befejezetlen'],
      dtype=object)

In [20]:
df_work['YEAR_BUILT'].unique() 

array(['1950 előtt', '1950-1980 között', nan, '2019', '2017', '2020',
       '2001-2010 között', '2015', '2018', '2012', '1981-2000 között',
       '2021', '2011', '2016', '2022', '2013', '2014'], dtype=object)

In [21]:
df_work['PRICE_PER_METER'] = df_work['PRICE'] / df_work['SIZE']

In [22]:
df_work.head(20)

,ID,SIZE,PRICE,ROOMS,FLOOR,ADDRESS,QUALITY,YEAR_BUILT,FLOORS_IN_BUILDING,LIFT,...,ORIENTATION,VIEW,METRO_NEARBY,PARKING,DISTRICT,LATITUDE,LONGITUDE,rooms,half_rooms,PRICE_PER_METER
0,23545892,39,31.9,1,0,"7. kerület, Dob utca 50.",közepes állapotú,1950 előtt,2,nincs megadva,...,dél,utcai,False,"utca, közterület - fizetős övezet",7,47.501126,19.065041,1,0,0.817949
1,23652121,79,65.9,3,4,"6. kerület, Belső-Terézváros",felújított,1950 előtt,nincs megadva,nincs megadva,...,nincs megadva,nincs megadva,False,nincs megadva,6,47.509494,19.065323,3,0,0.834177
2,23662271,75,89,2 + 1 fél,1,"7. kerület, Belső-Erzsébetváros",újszerű,1950 előtt,4,van,...,délnyugat,utcai,False,nincs megadva,7,47.502627,19.077243,2,1,1.18667
3,23662566,50,22.9,2,4,"3. kerület, Bécsi út",felújítandó,1950-1980 között,10,van,...,nyugat,utcai,False,"utca, közterület - fizetős övezet",3,47.532673,19.036491,2,0,0.458
4,23688806,32,12.5,1,3,"9. kerület, Haller utca",jó állapotú,NaN,3,nincs,...,nincs megadva,nincs megadva,False,"utca, közterület",9,47.476099,19.077198,1,0,0.390625
5,23722424,75,64.7,3,4,"13. kerület, The Gardens Budapest",új építésű,2019,5,van,...,kelet,utcai,False,teremgarázs hely - megvásárolható,13,47.533498,19.070615,3,0,0.862667
6,23745768,77,95,2,0,"6. kerület, Dózsa György út 102.",nincs megadva,NaN,nincs megadva,nincs megadva,...,nincs megadva,nincs megadva,False,nincs megadva,6,47.515406,19.075342,2,0,1.23377
7,23776860,45,42.9,2,1,"8. kerület, Víg utca",új építésű,2019,6,van,...,nyugat,utcai,False,teremgarázs hely - megvásárolható,8,47.493491,19.073544,2,0,0.953333
8,23792850,126,169.9,3 + 1 fél,4,"5. kerület, Zrínyi utca",felújítandó,NaN,6,van,...,délnyugat,panorámás,False,"utca, közterület",5,47.500191,19.048434,3,1,1.34841
9,23846400,62,75,2,0,"5. kerület, Károly körút 26.",nincs megadva,NaN,nincs megadva,nincs megadva,...,nincs megadva,nincs megadva,False,nincs megadva,5,47.496192,19.057291,2,0,1.20968


In [23]:
df_work.sort_values(by=['PRICE_PER_METER'])

,ID,SIZE,PRICE,ROOMS,FLOOR,ADDRESS,QUALITY,YEAR_BUILT,FLOORS_IN_BUILDING,LIFT,...,ORIENTATION,VIEW,METRO_NEARBY,PARKING,DISTRICT,LATITUDE,LONGITUDE,rooms,half_rooms,PRICE_PER_METER
10541,30933370,2222222,222220,222,0,"7. kerület, Rákóczi út",nincs megadva,NaN,nincs megadva,nincs megadva,...,nincs megadva,nincs megadva,False,nincs megadva,7,47.499044,19.077575,222,0,0.099999
10591,30937240,51,5.5,1 + 2 fél,5,"3. kerület, Hollós Korvin Lajos utca",jó állapotú,NaN,nincs megadva,van,...,nincs megadva,nincs megadva,False,nincs megadva,3,47.597314,19.050965,1,2,0.107843
16,23936899,65,8.65,1 + 2 fél,0,"9. kerület, Üllői út",közepes állapotú,1950 előtt,3,nincs,...,délkelet,utcai,False,"utca, közterület - fizetős övezet",9,47.473919,19.103220,1,2,0.133077
10657,30942235,41,5.5,1 + 1 fél,0,"14. kerület, Kacsóh Pongrác út",nincs megadva,NaN,nincs megadva,nincs megadva,...,nincs megadva,nincs megadva,False,nincs megadva,14,47.521993,19.092631,1,1,0.134146
3401,29829311,54,7.9,2,1,"22. kerület, Gyár utca",felújított,1950-1980 között,2,nincs,...,délnyugat,kertre néző,False,"utca, közterület - ingyenes",22,47.412131,19.038537,2,0,0.146296
16776,31297729,295,43.9,3,0,"14. kerület, Szabács utca",felújítandó,1950 előtt,2,nincs,...,nincs megadva,utcai,False,"utca, közterület - fizetős övezet",14,47.518549,19.115751,3,0,0.148814
12718,31082167,40,6,1,3,"13. kerület, Szent László út",jó állapotú,1950 előtt,4,nincs,...,nincs megadva,udvari,False,"utca, közterület",13,47.526986,19.085662,1,0,0.15
1263,28818701,120,20.5,3,1,"8. kerület, Józsefváros – Csarnoknegyed",közepes állapotú,1950 előtt,2,nincs megadva,...,kelet,utcai,False,"utca, közterület - fizetős övezet",8,47.488755,19.086433,3,0,0.170833
16018,31266850,35,5.99,1,0,"20. kerület, Hosszú utca",közepes állapotú,1950-1980 között,földszintes,nincs,...,nincs megadva,udvari,False,nincs megadva,20,47.444366,19.110940,1,0,0.171143
1385,28918940,114,19.59,2,-1,"13. kerület, Csongor utca",felújított,NaN,3,nincs megadva,...,nincs megadva,utcai,False,"utca, közterület",13,47.531918,19.071294,2,0,0.171842


## BKK

In [24]:
stops = pd.read_csv('./data/bkk/stops.txt')
stop_times = pd.read_csv('./data/bkk/stop_times.txt')
trips = pd.read_csv('./data/bkk/trips.txt')
routes = pd.read_csv('./data/bkk/routes.txt')

/Users/abudai/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [25]:
route_to_trip = trips[['route_id', 'trip_id']]
stop_to_trip = stop_times[['trip_id', 'stop_id']]

In [26]:
stops_filtered = stops[['stop_id', 'stop_name', 'stop_lat', 'stop_lon']]

In [27]:
all_stops = pd.merge(stop_to_trip.join(route_to_trip.set_index('trip_id'), on='trip_id'), stops_filtered, how='left', on='stop_id')

In [28]:
all_stops = all_stops.drop(columns=['trip_id', 'stop_id']).drop_duplicates()

In [29]:
routes[routes['route_short_name'] == 'M2']

,agency_id,route_id,route_short_name,route_long_name,route_type,route_desc,route_color,route_text_color,route_sort_order
291,BKK,5200,M2,NaN,1,Déli pályaudvar M / Örs vezér tere M+H,E41F18,FFFFFF,1


In [30]:
all_stops[all_stops['route_id'] == '5200']

,route_id,stop_name,stop_lat,stop_lon
0,5200,Déli pályaudvar,47.500450,19.024604
1,5200,Széll Kálmán tér,47.507533,19.026097
2,5200,Batthyány tér,47.506826,19.038103
3,5200,Kossuth Lajos tér,47.505511,19.045769
4,5200,Deák Ferenc tér,47.497593,19.053857
5,5200,Astoria,47.494825,19.062457
6,5200,Blaha Lujza tér,47.497010,19.070279
7,5200,Keleti pályaudvar,47.499683,19.082881
8,5200,Puskás Ferenc Stadion,47.500174,19.106381
9,5200,Pillangó utca,47.501204,19.120417


In [31]:
df_work['PRICE_PER_METER'] = df_work['PRICE_PER_METER'].astype('float')

In [43]:
df_district = df_work.groupby(['DISTRICT']).mean().reset_index()

In [47]:
df_district['DISTRICT'] = df_district['DISTRICT'].str.zfill(3)

In [48]:
df_district

,DISTRICT,SIZE,FLOOR,METRO_NEARBY,LATITUDE,LONGITUDE,rooms,PRICE_PER_METER
0,001,81.196481,1.642229,False,47.499336,19.033896,2.472141,1.085583
1,010,58.378682,2.136045,False,47.480347,19.143771,2.019635,0.595186
2,011,70.960000,2.452121,False,47.465549,19.024245,2.395758,0.914148
3,012,81.021700,1.218807,False,47.498139,19.007367,2.477396,1.101311
4,013,64.712615,2.650376,False,47.535467,19.059412,2.269424,0.893450
5,014,62.992612,1.994627,False,47.516051,19.117682,2.122230,0.724830
6,015,55.525381,3.017766,False,47.547625,19.133986,1.862944,0.569562
7,016,69.768061,1.326996,False,47.517585,19.189080,2.342205,0.675869
8,017,68.984962,1.778195,False,47.477034,19.259608,2.413534,0.594475
9,018,58.310096,2.512019,False,47.435855,19.175882,1.788462,0.564236


### from https://github.com/integralvision/geo-data-hungary/tree/master/GeoJSON

In [87]:
#geojson.GeoJSON(open('./data/GeoJSON/l40-district/001-budapest-01-kerulet.geojson').read())
geojson = folium.GeoJson(data=(open("./data/budapest.geojson", "r")).read())

In [93]:
json = geojson.data

In [91]:

# create geodataframe and add centroids
gdf = gpd.GeoDataFrame.from_features(geojson.data)
gdf['centroid_lon'] = gdf['geometry'].centroid.x
gdf['centroid_lat'] = gdf['geometry'].centroid.y
gdf.crs = {'init' :'epsg:4326'}
gdf.head()

/Users/abudai/anaconda3/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


,geometry,ksh,name,osm_id,parent,type,type:hu,centroid_lon,centroid_lat
0,"MULTIPOLYGON (((19.02373 47.49557, 19.02374 47...",001,Budapest I. kerület,221984,"{'name': 'Budapest', 'type': 'county', 'type:h...",district,járás,19.035772,47.496961
1,"MULTIPOLYGON (((18.92617 47.54345, 18.92979 47...",002,Budapest II. kerület,221980,"{'name': 'Budapest', 'type': 'county', 'type:h...",district,járás,18.979611,47.542218
2,"MULTIPOLYGON (((18.97760 47.57392, 18.97721 47...",003,Budapest III. kerület,221976,"{'name': 'Budapest', 'type': 'county', 'type:h...",district,járás,19.033603,47.569975
3,"MULTIPOLYGON (((19.06720 47.56314, 19.06741 47...",004,Budapest IV. kerület,367963,"{'name': 'Budapest', 'type': 'county', 'type:h...",district,járás,19.095804,47.579622
4,"MULTIPOLYGON (((19.04202 47.51471, 19.04263 47...",005,Budapest V. kerület,1606103,"{'name': 'Budapest', 'type': 'county', 'type:h...",district,járás,19.050677,47.500354


In [97]:
(str(json))

"{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'name': 'Budapest I. kerület', 'type': 'district', 'type:hu': 'járás', 'ksh': '001', 'osm_id': '221984', 'parent': {'name': 'Budapest', 'type': 'county', 'type:hu': 'megye', 'ksh': '01', 'osm_id': '1244004'}}, 'geometry': {'type': 'MultiPolygon', 'coordinates': [[[[19.0237324, 47.4955698], [19.0237383, 47.4956111], [19.0237567, 47.4957388], [19.0238881, 47.4966568], [19.0239841, 47.4973277], [19.0241375, 47.4983997], [19.0241755, 47.4986655], [19.0241636, 47.4992325], [19.0241714, 47.5002162], [19.0241933, 47.5004775], [19.0242287, 47.5006168], [19.0243734, 47.5011365], [19.0244431, 47.5015104], [19.0244845, 47.5018721], [19.0244108, 47.5024734], [19.0243675, 47.5028267], [19.0243355, 47.5029804], [19.0242646, 47.5032096], [19.0242442, 47.5033739], [19.0242523, 47.5035325], [19.0243421, 47.504113], [19.0243762, 47.504355], [19.0244308, 47.5048886], [19.0244378, 47.5050037], [19.0245924, 47.5055753], [19.0247

In [100]:

# create a map
m = folium.Map(location=[np.median(df_work['LATITUDE'].tolist()),
                         np.median(df_work['LONGITUDE'].tolist())],
               tiles='Stamen Toner', zoom_start=12)

## add chloropleth layer
m.choropleth(
    geo_data=json,
    name='Average price',
    data=df_district,
    columns=['DISTRICT', 'PRICE_PER_METER'],
    fill_color='YlGn',
    key_on='feature.properties.ksh',
    legend_name='inc_tax_per_capita'
)  

# add markers with basic information
'''
fg = folium.FeatureGroup(name='District Info')
for lat, lon, val, name in zip(gdf_income_tax['centroid_lat'].tolist(),
                               gdf_income_tax['centroid_lon'].tolist(),
                               gdf_income_tax['inc_tax_per_capita'].tolist(),
                               gdf_income_tax['name'].tolist()):
    html = f"""
    <h2>{name}<\h2><br>
    <h4>Income Tax Per Capita: {int(round(val,0))} HUF <\h4>
    """
    fg.add_child(folium.Marker(location=[lat, lon], popup=html))

m.add_child(fg)

# enable layers to be turned in or out
'''

'\nfg = folium.FeatureGroup(name=\'District Info\')\nfor lat, lon, val, name in zip(gdf_income_tax[\'centroid_lat\'].tolist(),\n                               gdf_income_tax[\'centroid_lon\'].tolist(),\n                               gdf_income_tax[\'inc_tax_per_capita\'].tolist(),\n                               gdf_income_tax[\'name\'].tolist()):\n    html = f"""\n    <h2>{name}<\\h2><br>\n    <h4>Income Tax Per Capita: {int(round(val,0))} HUF <\\h4>\n    """\n    fg.add_child(folium.Marker(location=[lat, lon], popup=html))\n\nm.add_child(fg)\n\n# enable layers to be turned in or out\n'

In [101]:
folium.LayerControl().add_to(m)

In [102]:
type(m)

folium.folium.Map

In [103]:
display(m)

In [ ]:
1